<h2>Utilisation de sqlite3

<h3>Importation, Connection, Curseur

In [27]:
import sqlite3
connexion=sqlite3.connect('example.db')
curseur=connexion.cursor()


## type des champs

SQLite	Python  
NULL	None  
INTEGER	int  
REAL	float  
TEXT	str par défaut  
BLOB	bytes  

In [28]:
# Create table
curseur.execute('''CREATE TABLE IF NOT EXISTS scores(
    id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
    pseudo TEXT,
    valeur INTEGER)''')

## Insertion

In [30]:
donnees = [("toto", 1000), ("tata", 750), ("titi", 500)]
#Exécutions multiples
for donnee in donnees:
    curseur.execute('''INSERT INTO scores (pseudo, valeur) VALUES (?, ?)''', donnee)
connexion.commit()  #Ne pas oublier de valider les modifications

In [31]:
donnees = [("toto", 1000), ("tata", 750), ("titi", 500)]
#Exécutions multiples
curseur.executemany("INSERT INTO scores (pseudo, valeur) VALUES (?, ?)", donnees)
connexion.commit()  #Ne pas oublier de valider les modifications

## Executemany

In [ ]:
donnees = (
    {"psd" : "toto", "val" : 1000},
    {"psd" : "tata", "val" : 750},
    {"psd" : "titi", "val" : 500}
)
#Exécutions multiples
curseur.executemany("INSERT INTO scores (pseudo, valeur) VALUES (:psd, :val)", donnees)
connexion.commit()  #Ne pas oublier de valider les modifications

## Execute script

In [70]:
#Exécution d'un script
curseur.executescript("""

    DROP TABLE IF EXISTS scores;

    CREATE TABLE IF NOT EXISTS scores(
    id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
    pseudo TEXT,
    valeur INTEGER);
    
    INSERT INTO scores(pseudo, valeur) VALUES ("toto", 1000);
    INSERT INTO scores(pseudo, valeur) VALUES ("tata", 750);
    INSERT INTO scores(pseudo, valeur) VALUES ("titi", 500)
""")
connexion.commit()  #Ne pas oublier de valider les modifications
connexion.close()

## Autre exemple

In [71]:
import sqlite3

#Connexion
connexion = sqlite3.connect('basededonnees.db')

#Récupération d'un curseur
curseur = connexion.cursor()

#Création de la table scores
curseur.execute("""
    CREATE TABLE IF NOT EXISTS scores2(
    id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
    pseudo TEXT,
    valeur INTEGER);
    """)

#Suppression des éléments de scores
curseur.execute("""DELETE FROM scores2""")

#Préparation des données à ajouter
donnees = [
    ("toto", 1000),
    ("tata", 750),
    ("titi", 500),
    ("toto", 250),
    ("tata", 150),
    ("tete", 0)
    ]

#Insertion des données
curseur.executemany('''INSERT INTO scores2 (pseudo, valeur) VALUES (?, ?)''', donnees)

#Validation
connexion.commit()

#Parcours des enregistrements....

#Déconnexion
#connexion.close()

<h3>Recuperation des donnees par SELECT *

In [72]:
donnee = ("titi", )
curseur.execute("SELECT valeur FROM scores2 WHERE pseudo = ?", donnee)
print(curseur.fetchone())  #affiche "(500,)"


(500,)


In [73]:
donnee = ("tata", )
curseur.execute("SELECT valeur FROM scores2 WHERE pseudo = ?", donnee)
result = curseur.fetchone()
while result:
    print(result)
    result = curseur.fetchone()
#affiche "(750,)" puis "(150,)"

(750,)
(150,)


In [74]:
print(curseur.arraysize)  #Affiche "1"
donnee = (400, )

curseur.execute("SELECT pseudo FROM scores2 WHERE valeur > ?", donnee)
print(curseur.fetchmany())  #Affiche "[('toto',)]"
print(curseur.fetchmany())  #Affiche "[('tata',)]"

curseur.execute("SELECT pseudo FROM scores2 WHERE valeur > ?", donnee)
print(curseur.fetchmany(2)) #Affiche "[('toto',), ('tata',)]"

1
[('toto',)]
[('tata',)]
[('toto',), ('tata',)]


In [75]:
curseur.execute("SELECT * FROM scores2")
resultats = curseur.fetchall()
for resultat in resultats:
    print(resultat)

(7, 'toto', 1000)
(8, 'tata', 750)
(9, 'titi', 500)
(10, 'toto', 250)
(11, 'tata', 150)
(12, 'tete', 0)


In [76]:
curseur.execute("SELECT * FROM scores2")
for resultat in curseur:
    print(resultat)

(7, 'toto', 1000)
(8, 'tata', 750)
(9, 'titi', 500)
(10, 'toto', 250)
(11, 'tata', 150)
(12, 'tete', 0)


Tout d’abord, pour savoir si des modifications ont été apportées sans être validées, il suffit de récupérer la valeur de l’attribut in_transaction de notre objet de type Connection. En effet, celui-ci vaut True si c’est le cas et False sinon.

In [77]:
#modifications...
print(connexion.in_transaction)  #Affiche "True"
connexion.commit()
print(connexion.in_transaction)  #Affiche "False"

False
False


Connaitre le nombre de lignes impactées par une exécution
De même, pour connaître le nombre de lignes impactées par une exécution, il suffit d’utiliser l’attribut rowcount de notre objet de type Cursor. S’il n’y a eu aucune exécution ou que le nombre de de lignes ne peut pas être déterminé (comme pour une sélection par exemple), il vaut -1. De plus, pour les versions de SQLite antérieure à la 3.6.5, la valeur vaut 0 après une suppression totale des éléments d’une table.

Voici un exemple :

In [78]:
print(curseur.rowcount)  #Affiche "-1"

donnee = ("toto", 1000)
curseur.execute("INSERT INTO scores2 (pseudo, valeur) VALUES (?, ?)", donnee)
print(curseur.rowcount)  #Affiche "1"

donnees = [("tata", 750), ("titi", 500)]
curseur.executemany("INSERT INTO scores2 (pseudo, valeur) VALUES (?, ?)", donnees)
print(curseur.rowcount)  #Affiche "2"

curseur.execute("SELECT * FROM scores2")
print(curseur.rowcount)  #Affiche "-1"

curseur.execute("DELETE FROM scores2")
print(curseur.rowcount)  #Affiche "-3" (0 si version SQLite < 3.6.5)

-1
1
2
-1
9


Récupérer l’identifiant de la dernière ligne insérée
Par ailleurs, nous pouvons aussi récupérer l’identifiant du dernier enregistrement dans une table à l’aide de l’attribut lastrowid de notre objet de type Connection :

In [79]:
from random import randint

#...

donnee = (randint(1, 1000), "toto", 1000)
print(donnee[0])  #Affiche "589"
curseur.execute("INSERT INTO scores2 (id, pseudo, valeur) VALUES (?, ?, ?)", donnee)
curseur.execute("SELECT * FROM scores2 WHERE id = ?", (curseur.lastrowid, ))
print(curseur.fetchone())  #Affiche "(589, 'toto', 1000)"
connexion.close()

409
(409, 'toto', 1000)


# Utiliser des clefs étrangères

In [80]:
import sqlite3

#Connexion
connexion = sqlite3.connect("basededonnees.db") 

#Récupération d'un curseur
curseur = connexion.cursor()

#Activation clés étrangères
curseur.execute("PRAGMA foreign_keys = ON") 

#Création table joueur puis score si elles n'existent pas encore
#Puis suppression des données dans joueurs (et dans scores aussi par cascade)
#afin d'éviter les répétitions d'enregistrements avec des exécutions multiples
curseur.executescript("""

    CREATE TABLE IF NOT EXISTS joueurs(
    id_joueur INTEGER PRIMARY KEY,
    pseudo TEXT,
    mdp TEXT);

    CREATE TABLE IF NOT EXISTS scores3(
    id_score INTEGER PRIMARY KEY,
    fk_joueur INTEGER NOT NULL,
    valeur INTEGER,
    FOREIGN KEY(fk_joueur) REFERENCES joueurs(id_joueur)
    ON DELETE CASCADE);

    DELETE FROM joueurs;
""")

#Préparation des données
donnees_joueur = [
    ("toto", "123"),
    ("tata", "azerty"),
    ("titi", "qwerty")
]
donnees_score = [
    (1, 1000),
    (2, 750),
    (3, 500)
]

#Insertion des données dans table joueur puis score
curseur.executemany("INSERT INTO joueurs (pseudo, mdp) VALUES (?, ?)", donnees_joueur)
curseur.executemany("INSERT INTO scores3 (fk_joueur, valeur) VALUES (?, ?)", donnees_score)

#Validation des ajouts
connexion.commit()

#Affichage des données
for joueur in curseur.execute("SELECT * FROM joueurs"):
    print("joueur :", joueur)

for score in curseur.execute("SELECT * FROM scores"):
    print("score :", score)


joueur : (1, 'toto', '123')
joueur : (2, 'tata', 'azerty')
joueur : (3, 'titi', 'qwerty')
score : (1, 'toto', 1000)
score : (2, 'tata', 750)
score : (3, 'titi', 500)


In [81]:
#Récupération du meilleur score
curseur.execute("""SELECT j.pseudo, s.valeur FROM joueurs as j INNER JOIN
    scores3 as s ON j.id_joueur = s.fk_joueur
    ORDER BY s.valeur DESC LIMIT 1""")
print(curseur.fetchone())  #Affiche "('toto', 1000)"

('toto', 1000)


In [82]:
connexion.close()

In [ ]:
e